In [ ]:
!mkdir data

In [ ]:
!mkdir data/training

In [ ]:
# Download validation
!gdown 1vkDgeFQ4Y8bK76W3TCqvrvSe_d0USwlT -O data/validation.zip
!unzip data/validation.zip -d data
!rm data/validation.zip

In [ ]:
# Download first part of stego
!gdown 1UKz2Vr_HdvlcGoB0GGaw6RzofT1gmj71 -O data/training/stego1.zip
!unzip data/training/stego1.zip -d data/training
!rm data/training/stego1.zip

In [ ]:
# Download second part of stego
!gdown 1hcf8ugegfRqZFCFy1huyA0fweCPhtgwG -O data/training/stego2.zip
!unzip data/training/stego2.zip -d data/training
!rm data/training/stego2.zip
!cp data/training/Stego2/*.png data/training/Stego
!rm -rf data/training/Stego2

In [ ]:
# Download cover
!gdown 12kuJFieXGE7frVNsGhprfUNswmb8Nlsl -O data/training/cover.zip
!unzip data/training/cover.zip  -d data/training
!rm data/training/cover.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/training/Cover/74673.jpg  
  inflating: data/training/Cover/74675.jpg  
  inflating: data/training/Cover/74676.jpg  
  inflating: data/training/Cover/74677.jpg  
  inflating: data/training/Cover/74678.jpg  
  inflating: data/training/Cover/74680.jpg  
  inflating: data/training/Cover/74681.jpg  
  inflating: data/training/Cover/74682.jpg  
  inflating: data/training/Cover/74683.jpg  
  inflating: data/training/Cover/74684.jpg  
  inflating: data/training/Cover/74685.jpg  
  inflating: data/training/Cover/74686.jpg  
  inflating: data/training/Cover/74687.jpg  
  inflating: data/training/Cover/74688.jpg  
  inflating: data/training/Cover/74689.jpg  
  inflating: data/training/Cover/74690.jpg  
  inflating: data/training/Cover/74691.jpg  
  inflating: data/training/Cover/74692.jpg  
  inflating: data/training/Cover/74693.jpg  
  inflating: data/training/Cover/74694.jpg  
  inflating: data/training/Cover/74695.jpg  
  in

In [ ]:
import os
cover_source_dir = "Cover"
juni_source_dir = "Stego"

training_path = "data/training"
validation_path = "data/validation"

training_cover_dir = os.path.join(training_path, "Cover")
validation_cover_dir = os.path.join(validation_path, "Cover")

training_juni_dir = os.path.join(training_path, "Stego")
validation_juni_dir = os.path.join(validation_path, "Stego")


In [ ]:
print(f"There are {len(os.listdir(training_cover_dir))} Cover images for training")
print(f"There are {len(os.listdir(training_juni_dir))} Stego images for training")

print(f"There are {len(os.listdir(validation_cover_dir))} Cover images for validation")
print(f"There are {len(os.listdir(validation_juni_dir))} Stego images for validation")

There are 60000 Cover images for training
There are 60000 Stego images for training
There are 15000 Cover images for validation
There are 15000 Stego images for validation


In [ ]:
from keras import mixed_precision

mixed_precision.set_global_policy('mixed_float16')

In [ ]:
training_batch_size = 32
validation_batch_size = 32

In [ ]:
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input


train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

train_generator = train_datagen.flow_from_directory(
    training_path,
    target_size = (512, 512),
    batch_size = training_batch_size,
    class_mode = "binary")

Found 120000 images belonging to 2 classes.


In [ ]:
validation_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size = (512, 512),
    batch_size = validation_batch_size,
    class_mode = "binary")

Found 30000 images belonging to 2 classes.


In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
%pip install efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.0 MB/s eta 0:00:00


In [ ]:
import efficientnet.tfkeras as efn

# B7 seems to be the best!
pretrained_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(
    include_top = False,
    pooling = "avg",
    classes = 2,
    weights = "imagenet")

for layer in pretrained_model.layers:
    layer.trainable = True


x = pretrained_model.output
output = tf.keras.layers.Dense(1, activation = "sigmoid", dtype = "float32")(x)   # May need dtype = float32

model = tf.keras.Model(inputs=pretrained_model.input, outputs=output)

24274472/24274472 [==============================] - 0s 0us/step


In [ ]:
optimizer = tf.keras.optimizers.AdamW(learning_rate = 0.001, weight_decay = 0.01)

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = "val_loss",
    patience = 2,
    factor = 0.5
)

In [ ]:
model.compile(
    loss="binary_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"]
)

In [ ]:
# Checkpoint to save best model seen so far
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('model-{epoch:03d}.keras', monitor='val_loss', save_best_only=False, mode='auto', save_weights_only = False)

In [ ]:
"""
!gdown 1n7VFHaNLmyNpHSf5-jhuOocDkUvYmark
"""

'\n!gdown 1n7VFHaNLmyNpHSf5-jhuOocDkUvYmark\n'

In [ ]:
"""
from tensorflow.keras.saving import load_model

model = load_model("model-007.h5")
"""

'\nfrom tensorflow.keras.saving import load_model\n\nmodel = load_model("model-007.h5")\n'

In [ ]:
EPOCHS = 50

history = model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data = validation_generator,
    callbacks = [checkpoint, reduce_lr]
)

Epoch 1/50
3750/3750 [==============================] - 2908s 751ms/step - loss: 0.4447 - accuracy: 0.7350 - val_loss: 0.4139 - val_accuracy: 0.7541 - lr: 0.0010
Epoch 2/50
3750/3750 [==============================] - 2780s 741ms/step - loss: 0.3871 - accuracy: 0.7651 - val_loss: 0.3916 - val_accuracy: 0.7414 - lr: 0.0010
Epoch 3/50
3750/3750 [==============================] - 2701s 720ms/step - loss: 0.3770 - accuracy: 0.7703 - val_loss: 0.3831 - val_accuracy: 0.7413 - lr: 0.0010
Epoch 4/50
3750/3750 [==============================] - 2685s 716ms/step - loss: 0.3717 - accuracy: 0.7710 - val_loss: 0.3784 - val_accuracy: 0.7445 - lr: 0.0010
Epoch 5/50
3750/3750 [==============================] - 2729s 728ms/step - loss: 0.3689 - accuracy: 0.7712 - val_loss: 0.3823 - val_accuracy: 0.7441 - lr: 0.0010
Epoch 6/50
3750/3750 [==============================] - 2686s 716ms/step - loss: 0.3659 - accuracy: 0.7725 - val_loss: 0.3755 - val_accuracy: 0.7848 - lr: 0.0010
Epoch 7/50
3750/3750 [======

In [ ]:
print("YOyoyoyo")

In [ ]:
print("AYO")

In [ ]:
print("ahdjkashdaskjhd")

In [ ]:
print("Yoyoyoyoyoyo")

In [ ]:
print("Hello World")